In [ ]:
import warnings
warnings.filterwarnings('ignore')
import glob
import pandas as pd
import numpy as np
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder
import random

In [ ]:
!pip install category_encoders
import category_encoders as ce

In [ ]:
import copy

In [ ]:
train_df = pd.read_csv('train.csv')
train_df = train_df.drop(['index'], axis=1)
train_df.fillna('NAN', inplace=True) 


test_df = pd.read_csv('test.csv')
test_df = test_df.drop(['index'], axis=1)
test_df.fillna('NAN', inplace=True)

submit = pd.read_csv('sample_submission.csv')

In [ ]:
object_col = []
for col in train_df.columns:
    if train_df[col].dtype == 'object':
        object_col.append(col)

In [ ]:
for i in object_col:
  print("{} : {}".format(i,len(train_df[i].unique())))

gender : 2
car : 2
reality : 2
income_type : 5
edu_type : 5
family_type : 5
house_type : 6
occyp_type : 19


In [ ]:
object_col

['gender',
 'car',
 'reality',
 'income_type',
 'edu_type',
 'family_type',
 'house_type',
 'occyp_type']

In [ ]:
X=train_df.drop("credit",axis=1).reset_index(drop=True)
y=pd.DataFrame(train_df["credit"]).reset_index(drop=True)

#onehot_cols=['gender','car','reality','income_type','edu_type','family_type']
onehot_cols=object_col
enc = ce.OneHotEncoder(cols=onehot_cols)
train_onehot=enc.fit_transform(X)
test_onehot =enc.transform(test_df)



# train_onehot_df = pd.DataFrame(enc.transform(train.loc[:,object_col]).toarray(), 
#              columns=enc.get_feature_names(object_col))
# train.drop(object_col, axis=1, inplace=True)
# train = pd.concat([train, train_onehot_df], axis=1)



In [ ]:
print(train_onehot)

       gender_1  gender_2  car_1  ...  occyp_type_19  family_size  begin_month
0             1         0      1  ...              0          2.0         -6.0
1             1         0      1  ...              0          3.0         -5.0
2             0         1      0  ...              0          2.0        -22.0
3             1         0      1  ...              0          2.0        -37.0
4             1         0      0  ...              0          2.0        -26.0
...         ...       ...    ...  ...            ...          ...          ...
26452         1         0      1  ...              0          4.0         -2.0
26453         1         0      1  ...              0          2.0        -47.0
26454         1         0      0  ...              0          2.0        -25.0
26455         0         1      1  ...              0          1.0        -59.0
26456         1         0      1  ...              0          2.0         -9.0

[26457 rows x 56 columns]


In [ ]:


train=pd.concat([train_onehot,y],axis=1)

In [ ]:
train

,gender_1,gender_2,car_1,car_2,reality_1,reality_2,child_num,income_total,income_type_1,income_type_2,income_type_3,income_type_4,income_type_5,edu_type_1,edu_type_2,edu_type_3,edu_type_4,edu_type_5,family_type_1,family_type_2,family_type_3,family_type_4,family_type_5,house_type_1,house_type_2,house_type_3,house_type_4,house_type_5,house_type_6,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type_1,occyp_type_2,occyp_type_3,occyp_type_4,occyp_type_5,occyp_type_6,occyp_type_7,occyp_type_8,occyp_type_9,occyp_type_10,occyp_type_11,occyp_type_12,occyp_type_13,occyp_type_14,occyp_type_15,occyp_type_16,occyp_type_17,occyp_type_18,occyp_type_19,family_size,begin_month,credit
0,1,0,1,0,1,0,0,202500.0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,-13899,-4709,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,-6.0,1.0
1,1,0,1,0,0,1,1,247500.0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,-11380,-1540,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.0,-5.0,1.0
2,0,1,0,1,0,1,0,450000.0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,-19087,-4434,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,-22.0,2.0
3,1,0,1,0,0,1,0,202500.0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,-15088,-2092,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,-37.0,0.0
4,1,0,0,1,0,1,0,157500.0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,-15037,-2105,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,-26.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26452,1,0,1,0,1,0,2,225000.0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,-12079,-1984,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,4.0,-2.0,1.0
26453,1,0,1,0,0,1,1,180000.0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,-15291,-2475,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,-47.0,2.0
26454,1,0,0,1,1,0,0,292500.0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,-10082,-2015,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,-25.0,2.0
26455,0,1,1,0,0,1,0,171000.0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,-10145,-107,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,-59.0,2.0


In [ ]:
from tqdm import tqdm_notebook as tqdm

In [ ]:
def makedata(train):
  a = np.arange(0, train.shape[1])
  #initialise aug dataframe - remember to set dtype!
  train_aug = pd.DataFrame(index=train.index, columns=train.columns, dtype='float64')

  for i in tqdm(range(0, len(train))):
      #ratio of features to be randomly sampled
      AUG_FEATURE_RATIO = 0.5
      #to integer count
      AUG_FEATURE_COUNT = np.floor(train.shape[1]*AUG_FEATURE_RATIO).astype('int16')
      
      #randomly sample half of columns that will contain random values
      aug_feature_index = np.random.choice(train.shape[1], AUG_FEATURE_COUNT, replace=False)
      aug_feature_index.sort()
      
      #obtain indices for features not in aug_feature_index
      feature_index = np.where(np.logical_not(np.in1d(a, aug_feature_index)))[0]
          
      #first insert real values for features in feature_index
      train_aug.iloc[i, feature_index] = train.iloc[i, feature_index]
                
      #random row index to randomly sampled values for each features
      rand_row_index = np.random.choice(len(train), len(aug_feature_index), replace=True)
          
      #for each feature being randomly sampled, extract value from random row in train
      for n, j in enumerate(aug_feature_index):
          train_aug.iloc[i, j] = train.iloc[rand_row_index[n], j]
  return train_aug



In [ ]:
def makedata2(train):
  a = np.arange(0, train.shape[1])
  #initialise aug dataframe - remember to set dtype!
  train_aug = pd.DataFrame(index=train.index, columns=train.columns, dtype='float64')
  train_aug2=pd.DataFrame(index=train.index, columns=train.columns, dtype='float64')

  for i in tqdm(range(0, len(train))):
      #ratio of features to be randomly sampled
      AUG_FEATURE_RATIO = 0.5
      #to integer count
      AUG_FEATURE_COUNT = np.floor(train.shape[1]*AUG_FEATURE_RATIO).astype('int16')

      #randomly sample half of columns that will contain random values
      aug_feature_index = np.random.choice(train.shape[1], AUG_FEATURE_COUNT, replace=False)
      aug_feature_index2 = np.random.choice(train.shape[1], AUG_FEATURE_COUNT, replace=False)
      aug_feature_index.sort()
      aug_feature_index2.sort()
      
      #obtain indices for features not in aug_feature_index
      feature_index = np.where(np.logical_not(np.in1d(a, aug_feature_index)))[0]
      feature_index2 = np.where(np.logical_not(np.in1d(a, aug_feature_index2)))[0]
          
      #first insert real values for features in feature_index
      train_aug.iloc[i, feature_index] = train.iloc[i, feature_index]
      train_aug2.iloc[i, feature_index2] = train.iloc[i, feature_index2]
                
      #random row index to randomly sampled values for each features
      rand_row_index = np.random.choice(len(train), len(aug_feature_index), replace=True)
      rand_row_index2 = np.random.choice(len(train), len(aug_feature_index2), replace=True)
          
      #for each feature being randomly sampled, extract value from random row in train
      for n, j in enumerate(aug_feature_index):
          train_aug.iloc[i, j] = train.iloc[rand_row_index[n], j]
      for n, j in enumerate(aug_feature_index2):
          train_aug2.iloc[i, j] = train.iloc[rand_row_index2[n], j]
  return train_aug,train_aug2


In [ ]:
temp_train1,temp_train2=makedata2(train)

In [ ]:
temp_train1

,gender_1,gender_2,car_1,car_2,reality_1,reality_2,child_num,income_total,income_type_1,income_type_2,income_type_3,income_type_4,income_type_5,edu_type_1,edu_type_2,edu_type_3,edu_type_4,edu_type_5,family_type_1,family_type_2,family_type_3,family_type_4,family_type_5,house_type_1,house_type_2,house_type_3,house_type_4,house_type_5,house_type_6,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type_1,occyp_type_2,occyp_type_3,occyp_type_4,occyp_type_5,occyp_type_6,occyp_type_7,occyp_type_8,occyp_type_9,occyp_type_10,occyp_type_11,occyp_type_12,occyp_type_13,occyp_type_14,occyp_type_15,occyp_type_16,occyp_type_17,occyp_type_18,occyp_type_19,family_size,begin_month,credit
0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,202500.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-15857.0,-2135.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,-58.0,1.0
1,1.0,0.0,1.0,0.0,0.0,0.0,0.0,247500.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,-10809.0,-2346.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,-5.0,1.0
2,1.0,1.0,1.0,1.0,0.0,1.0,0.0,450000.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-19087.0,-4434.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,-27.0,2.0
3,1.0,0.0,0.0,0.0,0.0,1.0,0.0,157500.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,-21225.0,365243.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,-21.0,0.0
4,1.0,0.0,0.0,1.0,0.0,1.0,0.0,90000.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-15516.0,-1945.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,-4.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26452,1.0,1.0,0.0,1.0,0.0,1.0,1.0,103500.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-17138.0,365243.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,-14.0,2.0
26453,1.0,0.0,1.0,0.0,0.0,0.0,1.0,360000.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-15552.0,-4184.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,-47.0,2.0
26454,1.0,0.0,1.0,0.0,1.0,0.0,0.0,292500.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-10082.0,-2015.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,-29.0,2.0
26455,1.0,1.0,1.0,0.0,0.0,1.0,1.0,171000.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-15725.0,-2691.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-9.0,2.0


In [ ]:
temp_train1

,gender_1,gender_2,car_1,car_2,reality_1,reality_2,child_num,income_total,income_type_1,income_type_2,income_type_3,income_type_4,income_type_5,edu_type_1,edu_type_2,edu_type_3,edu_type_4,edu_type_5,family_type_1,family_type_2,family_type_3,family_type_4,family_type_5,house_type_1,house_type_2,house_type_3,house_type_4,house_type_5,house_type_6,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type_1,occyp_type_2,occyp_type_3,occyp_type_4,occyp_type_5,occyp_type_6,occyp_type_7,occyp_type_8,occyp_type_9,occyp_type_10,occyp_type_11,occyp_type_12,occyp_type_13,occyp_type_14,occyp_type_15,occyp_type_16,occyp_type_17,occyp_type_18,occyp_type_19,family_size,begin_month,credit
0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,202500.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-15857.0,-2135.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,-58.0,1.0
1,1.0,0.0,1.0,0.0,0.0,0.0,0.0,247500.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,-10809.0,-2346.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,-5.0,1.0
2,1.0,1.0,1.0,1.0,0.0,1.0,0.0,450000.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-19087.0,-4434.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,-27.0,2.0
3,1.0,0.0,0.0,0.0,0.0,1.0,0.0,157500.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,-21225.0,365243.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,-21.0,0.0
4,1.0,0.0,0.0,1.0,0.0,1.0,0.0,90000.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-15516.0,-1945.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,-4.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26452,1.0,1.0,0.0,1.0,0.0,1.0,1.0,103500.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-17138.0,365243.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,-14.0,2.0
26453,1.0,0.0,1.0,0.0,0.0,0.0,1.0,360000.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-15552.0,-4184.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,-47.0,2.0
26454,1.0,0.0,1.0,0.0,1.0,0.0,0.0,292500.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-10082.0,-2015.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,-29.0,2.0
26455,1.0,1.0,1.0,0.0,0.0,1.0,1.0,171000.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-15725.0,-2691.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-9.0,2.0


In [ ]:
a = np.arange(0, train.shape[1])
#initialise aug dataframe - remember to set dtype!
train_aug = pd.DataFrame(index=train.index, columns=train.columns, dtype='float64')

for i in tqdm(range(0, len(train))):
    #ratio of features to be randomly sampled
    AUG_FEATURE_RATIO = 0.5
    #to integer count
    AUG_FEATURE_COUNT = np.floor(train.shape[1]*AUG_FEATURE_RATIO).astype('int16')
    
    #randomly sample half of columns that will contain random values
    aug_feature_index = np.random.choice(train.shape[1], AUG_FEATURE_COUNT, replace=False)
    aug_feature_index.sort()
    
    #obtain indices for features not in aug_feature_index
    feature_index = np.where(np.logical_not(np.in1d(a, aug_feature_index)))[0]
        
    #first insert real values for features in feature_index
    train_aug.iloc[i, feature_index] = train.iloc[i, feature_index]
              
    #random row index to randomly sampled values for each features
    rand_row_index = np.random.choice(len(train), len(aug_feature_index), replace=True)
        
    #for each feature being randomly sampled, extract value from random row in train
    for n, j in enumerate(aug_feature_index):
        train_aug.iloc[i, j] = train.iloc[rand_row_index[n], j]

In [ ]:
train_aug

,gender_1,gender_2,car_1,car_2,reality_1,reality_2,child_num,income_total,income_type_1,income_type_2,income_type_3,income_type_4,income_type_5,edu_type_1,edu_type_2,edu_type_3,edu_type_4,edu_type_5,family_type_1,family_type_2,family_type_3,family_type_4,family_type_5,house_type_1,house_type_2,house_type_3,house_type_4,house_type_5,house_type_6,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type_1,occyp_type_2,occyp_type_3,occyp_type_4,occyp_type_5,occyp_type_6,occyp_type_7,occyp_type_8,occyp_type_9,occyp_type_10,occyp_type_11,occyp_type_12,occyp_type_13,occyp_type_14,occyp_type_15,occyp_type_16,occyp_type_17,occyp_type_18,occyp_type_19,family_size,begin_month,credit
0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,202500.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-13899.0,-433.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,-24.0,2.0
1,1.0,0.0,1.0,0.0,0.0,1.0,1.0,90000.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-11025.0,-2860.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,-52.0,1.0
2,0.0,0.0,0.0,1.0,1.0,1.0,0.0,135000.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-14080.0,-4434.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,-13.0,2.0
3,1.0,0.0,0.0,1.0,0.0,0.0,0.0,135000.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-11785.0,-8448.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,-6.0,0.0
4,1.0,0.0,0.0,0.0,0.0,1.0,0.0,112500.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-15037.0,-2105.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,-26.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26452,0.0,0.0,1.0,0.0,1.0,1.0,2.0,270000.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-12079.0,-1984.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,-2.0,0.0
26453,0.0,0.0,1.0,0.0,0.0,1.0,1.0,180000.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-15291.0,-513.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,-17.0,2.0
26454,0.0,0.0,1.0,1.0,1.0,0.0,0.0,292500.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,-10082.0,-229.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,-6.0,2.0
26455,0.0,0.0,0.0,1.0,0.0,1.0,2.0,171000.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-10145.0,-107.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-19.0,2.0


In [ ]:
train_all = pd.concat([train, train_aug])
y_all = np.append(y, y)
train_all=train_all.reset_index(drop=True)


print('Original train data shape: {}'.format(train.shape))
print('Augmented train data shape: {}'.format(train_all.shape))



Original train data shape: (26457, 57)
Augmented train data shape: (52914, 57)


In [ ]:
train_double = pd.concat([train, train])
train_double=train_double.reset_index(drop=True)
#y_all = np.append(y, y)

print('Original train data shape: {}'.format(train.shape))
print('Augmented train data shape: {}'.format(train_double.shape))

Original train data shape: (26457, 57)
Augmented train data shape: (52914, 57)


In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

def make_idx(train_df):
  folds=[]
  for train_idx, valid_idx in skf.split(train_df, train_df['credit']):
    folds.append((train_idx, valid_idx))
  return folds

# for train_idx, valid_idx in skf.split(train_df, train_df['credit']):
#     folds.append((train_idx, valid_idx))

In [ ]:
folds=make_idx(train_all)

In [ ]:
fold_normal=make_idx(train)

In [ ]:
train

,gender_1,gender_2,car_1,car_2,reality_1,reality_2,child_num,income_total,income_type_1,income_type_2,income_type_3,income_type_4,income_type_5,edu_type_1,edu_type_2,edu_type_3,edu_type_4,edu_type_5,family_type_1,family_type_2,family_type_3,family_type_4,family_type_5,house_type_1,house_type_2,house_type_3,house_type_4,house_type_5,house_type_6,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type_1,occyp_type_2,occyp_type_3,occyp_type_4,occyp_type_5,occyp_type_6,occyp_type_7,occyp_type_8,occyp_type_9,occyp_type_10,occyp_type_11,occyp_type_12,occyp_type_13,occyp_type_14,occyp_type_15,occyp_type_16,occyp_type_17,occyp_type_18,occyp_type_19,family_size,begin_month,credit
0,1,0,1,0,1,0,0,202500.0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,-13899,-4709,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,-6.0,1.0
1,1,0,1,0,0,1,1,247500.0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,-11380,-1540,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.0,-5.0,1.0
2,0,1,0,1,0,1,0,450000.0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,-19087,-4434,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,-22.0,2.0
3,1,0,1,0,0,1,0,202500.0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,-15088,-2092,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,-37.0,0.0
4,1,0,0,1,0,1,0,157500.0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,-15037,-2105,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,-26.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26452,1,0,1,0,1,0,2,225000.0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,-12079,-1984,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,4.0,-2.0,1.0
26453,1,0,1,0,0,1,1,180000.0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,-15291,-2475,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,-47.0,2.0
26454,1,0,0,1,1,0,0,292500.0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,-10082,-2015,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,-25.0,2.0
26455,0,1,1,0,0,1,0,171000.0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,-10145,-107,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,-59.0,2.0


In [ ]:
random.seed(42)
lgbm_params={'reg_alpha': 1.0997191680377813e-05, 'reg_lambda': 0.04104630401883339, 'max_depth': 15,
             'num_leaves': 191, 'colsample_bytree': 0.5198042692950159, 'subsample': 0.6599641068895281,
             'subsample_freq': 9, 'min_child_samples': 7, 'max_bin': 334, 'n_estimators':1000}

def cvdef(train_df,folds):



  lgb_models={}
  for fold in range(5):
      print(f'===================================={fold+1}============================================')
      train_idx, valid_idx = folds[fold]
      X_train, X_valid, y_train, y_valid = train_df.drop(['credit'],axis=1).iloc[train_idx].values, train_df.drop(['credit'],axis=1).iloc[valid_idx].values,\
                                          train_df['credit'][train_idx].values, train_df['credit'][valid_idx].values 
      lgb = LGBMClassifier(**lgbm_params)
      lgb.fit(X_train, y_train, 
              eval_set=[(X_train, y_train), (X_valid, y_valid)], 
              early_stopping_rounds=100,
            verbose=100)
      lgb_models[fold]=lgb
      print(f'================================================================================\n\n')
  return lgb_models

In [ ]:
lgb_model_origin=cvdef(train,fold_normal)

====================================1============================================
Training until validation scores don't improve for 100 rounds.
[100]	training's multi_logloss: 0.42971	valid_1's multi_logloss: 0.710125
[200]	training's multi_logloss: 0.288284	valid_1's multi_logloss: 0.722417
Early stopping, best iteration is:
[109]	training's multi_logloss: 0.412764	valid_1's multi_logloss: 0.708132


====================================2============================================
Training until validation scores don't improve for 100 rounds.
[100]	training's multi_logloss: 0.43647	valid_1's multi_logloss: 0.721508
[200]	training's multi_logloss: 0.290738	valid_1's multi_logloss: 0.74444
Early stopping, best iteration is:
[111]	training's multi_logloss: 0.414514	valid_1's multi_logloss: 0.720707


====================================3============================================
Training until validation scores don't improve for 100 rounds.
[100]	training's multi_logloss: 0.426061	val

In [ ]:
# def objective(trial: Trial) -> float:
#     params_lgb = {
#         "random_state": 42,
#         "verbosity": -1,
#         "learning_rate": 0.05,
#         "n_estimators": 10000,
#         "objective": "multiclass",
#         "metric": "multi_logloss",
#         "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 3e-5),
#         "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 9e-2),
#         "max_depth": trial.suggest_int("max_depth", 1, 20),
#         "num_leaves": trial.suggest_int("num_leaves", 2, 256),
#         "colsample_bytree": trial.suggest_float("colsample_bytree", 0.4, 1.0),
#         "subsample": trial.suggest_float("subsample", 0.3, 1.0),
#         "subsample_freq": trial.suggest_int("subsample_freq", 1, 10),
#         "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
#         "max_bin": trial.suggest_int("max_bin", 200, 500),
#     }
    
#     X_train, X_valid, y_train, y_valid = train_test_split(X_objective, y_objective, test_size=0.2)

#     model = LGBMClassifier(**params_lgb)
#     model.fit(
#         X_train,
#         y_train,
#         eval_set=[(X_train, y_train), (X_valid, y_valid)],
#         early_stopping_rounds=100,
#         verbose=False,
#     )

#     lgb_pred = model.predict_proba(X_valid)
#     log_score = log_loss(y_valid, lgb_pred)
    
#     return log_score

In [ ]:
# X_objective

In [ ]:
# sampler = TPESampler(seed=42)
# study = optuna.create_study(
#     study_name="lgbm_parameter_opt",
#     direction="minimize",
#     sampler=sampler,
# )
# study.optimize(objective, n_trials=10)
# print("Best Score:", study.best_value)
# print("Best trial:", study.best_trial.params)

In [ ]:
lgbm_params={'reg_alpha': 1.0997191680377813e-05, 'reg_lambda': 0.04104630401883339, 'max_depth': 15,
             'num_leaves': 191, 'colsample_bytree': 0.5198042692950159, 'subsample': 0.6599641068895281,
             'subsample_freq': 9, 'min_child_samples': 7, 'max_bin': 334, 'n_estimators':1000}


def doubledef(train_df,folds):



  lgb_models={}
  for fold in range(5):
      print(f'===================================={fold+1}============================================')
      train_idx, valid_idx = folds[fold]
      X_train, X_valid, y_train, y_valid = train_df.drop(['credit'],axis=1).iloc[train_idx], train_df.drop(['credit'],axis=1).iloc[valid_idx].values,\
                                          train_df['credit'][train_idx], train_df['credit'][valid_idx].values 
          
      #X_train_double=np.concatenate((X_train, X_train), axis=0)

      #y_train_double=np.concatenate((y_train, y_train), axis=0)

      X_pd=pd.DataFrame(X_train)
      y_pd=pd.DataFrame(y_train)
      temp_pd=pd.concat([X_pd,y_pd],axis=1)

      aug_data=makedata(temp_pd)
      train_temp = pd.concat([temp_pd, aug_data])
      train_temp=train_temp.reset_index(drop=True)

      X_train_double=train_temp.drop(['credit'],axis=1).values
      y_train_double=train_temp['credit'].values
      


      lgb = LGBMClassifier(**lgbm_params)
      lgb.fit(X_train_double, y_train_double, 
              eval_set=[(X_train_double, y_train_double), (X_valid, y_valid)], 
              early_stopping_rounds=100,
            verbose=100)
      lgb_models[fold]=lgb
      print(f'================================================================================\n\n')
  return lgb_models

In [ ]:
lgbm_params={'reg_alpha': 1.0997191680377813e-05, 'reg_lambda': 0.04104630401883339, 'max_depth': 15,
             'num_leaves': 191, 'colsample_bytree': 0.5198042692950159, 'subsample': 0.6599641068895281,
             'subsample_freq': 9, 'min_child_samples': 7, 'max_bin': 334, 'n_estimators':1000}


def tripledef(train_df,folds):



  lgb_models={}
  for fold in range(5):
      print(f'===================================={fold+1}============================================')
      train_idx, valid_idx = folds[fold]
      X_train, X_valid, y_train, y_valid = train_df.drop(['credit'],axis=1).iloc[train_idx], train_df.drop(['credit'],axis=1).iloc[valid_idx].values,\
                                          train_df['credit'][train_idx], train_df['credit'][valid_idx].values 
          
      #X_train_double=np.concatenate((X_train, X_train), axis=0)

      #y_train_double=np.concatenate((y_train, y_train), axis=0)

      X_pd=pd.DataFrame(X_train)
      y_pd=pd.DataFrame(y_train)
      temp_pd=pd.concat([X_pd,y_pd],axis=1)

      aug_data=makedata(temp_pd)
      train_temp = pd.concat([temp_pd, aug_data])
      train_temp=train_temp.reset_index(drop=True)

      X_train_double=train_temp.drop(['credit'],axis=1).values
      y_train_double=train_temp['credit'].values
      


      lgb = LGBMClassifier(**lgbm_params)
      lgb.fit(X_train_double, y_train_double, 
              eval_set=[(X_train_double, y_train_double), (X_valid, y_valid)], 
              early_stopping_rounds=100,
            verbose=100)
      lgb_models[fold]=lgb
      print(f'================================================================================\n\n')
  return lgb_models

In [ ]:
train.columns

Index(['gender_1', 'gender_2', 'car_1', 'car_2', 'reality_1', 'reality_2',
       'child_num', 'income_total', 'income_type_1', 'income_type_2',
       'income_type_3', 'income_type_4', 'income_type_5', 'edu_type_1',
       'edu_type_2', 'edu_type_3', 'edu_type_4', 'edu_type_5', 'family_type_1',
       'family_type_2', 'family_type_3', 'family_type_4', 'family_type_5',
       'house_type_1', 'house_type_2', 'house_type_3', 'house_type_4',
       'house_type_5', 'house_type_6', 'DAYS_BIRTH', 'DAYS_EMPLOYED',
       'FLAG_MOBIL', 'work_phone', 'phone', 'email', 'occyp_type_1',
       'occyp_type_2', 'occyp_type_3', 'occyp_type_4', 'occyp_type_5',
       'occyp_type_6', 'occyp_type_7', 'occyp_type_8', 'occyp_type_9',
       'occyp_type_10', 'occyp_type_11', 'occyp_type_12', 'occyp_type_13',
       'occyp_type_14', 'occyp_type_15', 'occyp_type_16', 'occyp_type_17',
       'occyp_type_18', 'occyp_type_19', 'family_size', 'begin_month',
       'credit'],
      dtype='object')

In [ ]:
select_feature=['DAYS_BIRTH','DAYS_EMPLOYED','begin_month','income_total','credit']

In [ ]:
train_select=train[select_feature]

In [ ]:
lgb_models_select_2=cvdef(train_select,fold_normal)

====================================1============================================
Training until validation scores don't improve for 100 rounds.
[100]	training's multi_logloss: 0.569164	valid_1's multi_logloss: 0.752521
Early stopping, best iteration is:
[93]	training's multi_logloss: 0.577557	valid_1's multi_logloss: 0.751456


====================================2============================================
Training until validation scores don't improve for 100 rounds.
[100]	training's multi_logloss: 0.562045	valid_1's multi_logloss: 0.789621
Early stopping, best iteration is:
[67]	training's multi_logloss: 0.610336	valid_1's multi_logloss: 0.783947


====================================3============================================
Training until validation scores don't improve for 100 rounds.
[100]	training's multi_logloss: 0.565195	valid_1's multi_logloss: 0.774431
Early stopping, best iteration is:
[73]	training's multi_logloss: 0.602643	valid_1's multi_logloss: 0.772575


=======

In [ ]:
lgb_models_select=doubledef(train_select,fold_normal)

====================================1============================================



Training until validation scores don't improve for 100 rounds.
[100]	training's multi_logloss: 0.705387	valid_1's multi_logloss: 0.77827
[200]	training's multi_logloss: 0.640109	valid_1's multi_logloss: 0.775283
Early stopping, best iteration is:
[159]	training's multi_logloss: 0.6621	valid_1's multi_logloss: 0.773773


====================================2============================================



Training until validation scores don't improve for 100 rounds.
[100]	training's multi_logloss: 0.704761	valid_1's multi_logloss: 0.795275
[200]	training's multi_logloss: 0.63883	valid_1's multi_logloss: 0.800624
Early stopping, best iteration is:
[107]	training's multi_logloss: 0.698737	valid_1's multi_logloss: 0.794376


====================================3============================================



Training until validation scores don't improve for 100 rounds.
[100]	training's multi_logloss: 0.705191	valid_1's multi_logloss: 0.792757
[200]	training's multi_logloss: 0.638277	valid_1's multi_logloss: 0.799864
Early stopping, best iteration is:
[125]	training's multi_logloss: 0.685119	valid_1's multi_logloss: 0.791798


====================================4============================================



Training until validation scores don't improve for 100 rounds.
[100]	training's multi_logloss: 0.70892	valid_1's multi_logloss: 0.781982
[200]	training's multi_logloss: 0.644344	valid_1's multi_logloss: 0.781768
Early stopping, best iteration is:
[153]	training's multi_logloss: 0.670969	valid_1's multi_logloss: 0.777763


====================================5============================================



Training until validation scores don't improve for 100 rounds.
[100]	training's multi_logloss: 0.706398	valid_1's multi_logloss: 0.788872
[200]	training's multi_logloss: 0.639684	valid_1's multi_logloss: 0.79269
Early stopping, best iteration is:
[112]	training's multi_logloss: 0.696016	valid_1's multi_logloss: 0.788419




In [ ]:
lgb_models=doubledef(train,fold_normal)

====================================1============================================



Training until validation scores don't improve for 100 rounds.
[100]	training's multi_logloss: 0.590891	valid_1's multi_logloss: 0.739195
[200]	training's multi_logloss: 0.450561	valid_1's multi_logloss: 0.723463
[300]	training's multi_logloss: 0.355682	valid_1's multi_logloss: 0.72039
Early stopping, best iteration is:
[268]	training's multi_logloss: 0.383856	valid_1's multi_logloss: 0.719089


====================================2============================================



Training until validation scores don't improve for 100 rounds.
[100]	training's multi_logloss: 0.588251	valid_1's multi_logloss: 0.743163
[200]	training's multi_logloss: 0.446252	valid_1's multi_logloss: 0.730502
[300]	training's multi_logloss: 0.351868	valid_1's multi_logloss: 0.73353
Early stopping, best iteration is:
[219]	training's multi_logloss: 0.425861	valid_1's multi_logloss: 0.730417


====================================3============================================



Training until validation scores don't improve for 100 rounds.
[100]	training's multi_logloss: 0.592977	valid_1's multi_logloss: 0.746137
[200]	training's multi_logloss: 0.450892	valid_1's multi_logloss: 0.735348
[300]	training's multi_logloss: 0.353019	valid_1's multi_logloss: 0.738999
Early stopping, best iteration is:
[234]	training's multi_logloss: 0.414153	valid_1's multi_logloss: 0.735009


====================================4============================================



Training until validation scores don't improve for 100 rounds.
[100]	training's multi_logloss: 0.590355	valid_1's multi_logloss: 0.743689
[200]	training's multi_logloss: 0.449761	valid_1's multi_logloss: 0.7309
[300]	training's multi_logloss: 0.355381	valid_1's multi_logloss: 0.731725
Early stopping, best iteration is:
[224]	training's multi_logloss: 0.424232	valid_1's multi_logloss: 0.728255


====================================5============================================



Training until validation scores don't improve for 100 rounds.
[100]	training's multi_logloss: 0.593919	valid_1's multi_logloss: 0.743088
[200]	training's multi_logloss: 0.453195	valid_1's multi_logloss: 0.733914
[300]	training's multi_logloss: 0.356497	valid_1's multi_logloss: 0.74112
Early stopping, best iteration is:
[216]	training's multi_logloss: 0.435399	valid_1's multi_logloss: 0.733184




In [ ]:
# random.seed(42)
# lgb_models={}
# for fold in range(5):
#     print(f'===================================={fold+1}============================================')
#     train_idx, valid_idx = folds[fold]
#     X_train, X_valid, y_train, y_valid = train_df.drop(['credit'],axis=1).iloc[train_idx].values, train_df.drop(['credit'],axis=1).iloc[valid_idx].values,\
#                                          train_df['credit'][train_idx].values, train_df['credit'][valid_idx].values 
#     lgb = LGBMClassifier(n_estimators=1000)
#     lgb.fit(X_train, y_train, 
#             eval_set=[(X_train, y_train), (X_valid, y_valid)], 
#             early_stopping_rounds=30,
#            verbose=100)
#     lgb_models[fold]=lgb
#     print(f'================================================================================\n\n')

In [ ]:
submit.iloc[:,1:]=0
for fold in range(5):
    submit.iloc[:,1:] += lgb_models[fold].predict_proba(test_onehot)/5

In [ ]:
submit.iloc[:,1:]=0
for fold in range(5):
    submit.iloc[:,1:] += lgb_model_origin[fold].predict_proba(test_onehot)/5

In [ ]:
submit

,index,0,1,2
0,26457,0.069020,0.081577,0.849404
1,26458,0.197789,0.220266,0.581945
2,26459,0.042181,0.101210,0.856609
3,26460,0.116930,0.128745,0.754325
4,26461,0.044084,0.161505,0.794411
...,...,...,...,...
9995,36452,0.116247,0.264187,0.619566
9996,36453,0.190936,0.376060,0.433004
9997,36454,0.027281,0.090261,0.882458
9998,36455,0.128024,0.283991,0.587985


In [ ]:
from sklearn.metrics import accuracy_score,log_loss,precision_score

NameError: ignored

In [ ]:
submit.to_csv('20210524_foldhyper_submit_lightgbm.csv', index=False) # 0.7272812144